In [ ]:
#bigbasket dynamic products by scrolling

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import time
import csv
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Upgrade-Insecure-Requests': '1'
    }

In [ ]:
driver = webdriver.Edge()
wait = WebDriverWait(driver, 10)

def search_category():
    cat_in = input("").strip().lower()
    cat_ab = (
        "apparel","fruits-vegetables", "foodgrains-oil-masala", "bakery-cakes-dairy", "beverages",
        "snacks-branded-foods", "beauty-hygiene", "cleaning-household", "kitchen-garden-pets",
        "lunch-boxes-bags", "umbrellas-rainwear","eggs-meat-fish", "gourmet-world-food", "baby-care", "paan-corner"
    )
    for k in cat_ab:
        for j in k.split("-"):
            if cat_in in j:
                return k
    return None

def scrap_category(search):
    if not search:
        return []

    url = f"https://www.bigbasket.com/cl/{search}/?nc=nb"
    driver.get(url)
    # time.sleep(2)
    product_links = set()  
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True :
        driver.execute_script("window.scrollBy(0, window.innerHeight*1.4);")
        time.sleep(3)  # Adjust the sleep time if needed
        
        # Check if the page height has increased
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit if no new content is loaded
        last_height = new_height
        
    # Extract product links from the current view
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_cards = soup.find_all('div', class_="SKUDeck___StyledDiv-sc-1e5d9gk-0 eA-dmzP")
    for card in all_cards:
        link = card.find('a', rel="noopener noreferrer")
        if link:
            product_links.add(f"https://www.bigbasket.com{link.get('href')}")
            
    return list(product_links)


def scrap_product(url_page):
    try:
        driver.get(url_page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract brand, title, category, price, weight/volume, and images
        brand = soup.find('a', class_="Description___StyledLink-sc-82a36a-1 gePjxR")
        title = soup.find('h1', class_="Description___StyledH-sc-82a36a-2 bofYPK")
        category_elements = soup.find_all('span', class_="name text-md leading-md xl:leading-sm xl:text-base text-darkOnyx-800 false")
        price = soup.find('td', class_="Description___StyledTd-sc-82a36a-4 fLZywG")
        w_v = soup.find('span', class_="Label-sc-15v1nk5-0 PackSizeSelector___StyledLabel2-sc-l9rhbt-2 gJxZPQ hDJUsF")

        # Process extracted elements
        brand_text = brand.text.strip() if brand else None
        title_text = title.text.strip() if title else None
        price_text = price.text[8:].strip() if price else None
        w_v_text = w_v.text.strip() if w_v else None

        # Construct category path
        category = " -> ".join([elem.text for elem in category_elements]) if category_elements else None

        # Determine count, quantity, and unit from weight/volume text
        count = 1
        unit = ""
        if not w_v_text and title_text:
            sample = title_text.split()
            units = ["kg", "ml", "pc"]
            prev = ""
            for unit in units:
                for word in sample:
                    if unit in word:
                        w_v_text = prev + word
                        unit = word.strip()
                        break
                    prev = word
        
        if w_v_text:
            w_v_text = w_v_text.strip()
            unit = w_v_text[-2:].strip()
            if "x" in w_v_text:
                w_v_parts = w_v_text.split("x")
                count = w_v_parts[0].strip()
                if "pc" in w_v_parts[1]:
                    w_v_text = w_v_parts[1][:-3].strip()
                    unit = "pcs"
                else:
                    numbers = re.findall(r'\d+', w_v_parts[1])
                    alphabets = re.findall(r'[a-zA-Z]+', w_v_parts[1])
                    w_v_text = ' '.join(numbers)
                    unit = ' '.join(alphabets)
            else:
                if " " in w_v_text:
                    parts = w_v_text.split()
                    w_v_text = parts[0]
                    unit = parts[1]
                else:
                    numbers = re.findall(r'\d+', w_v_text)
                    alphabets = re.findall(r'[a-zA-Z]+', w_v_text)
                    w_v_text = ' '.join(numbers)
                    unit = ' '.join(alphabets)

        # Extract image links
        image_links = set()
        image_divs = soup.find_all('div',class_="relative h-full w-full")
        # print(images)
        for div in image_divs:
            img = div.find("img", lazyboundary="800px")
            if img:
                link = img.get('src')
                if link:
                    if not link.startswith('data:image/gif;'):
                        link = link.split("?")[0]
                        link = link.replace("/s/", "/l/")
                        image_links.add(link)

        # Extract "About the Product" and "Other Product Info" sections
        details = {"About the Product":None,"Other Product Info":None,"EAN Code":None}
        about_section = soup.find('span', string='About the Product')
        other_info_section = soup.find('span', string='Other Product Info')

        if about_section:
            about_content = about_section.find_next("div").find_next("div")
            details["About the Product"] = about_content.text.strip() if about_content else None

        if other_info_section:
            other_info_content = other_info_section.find_next("div").find_next("div")
            details["Other Product Info"] = other_info_content.text.strip() if other_info_content else None
            if "EAN" in details.get("Other Product Info", ""):
                ean_matches = re.findall(r'\d+', details["Other Product Info"])
                details["EAN Code"] = ean_matches[0] if ean_matches else None

        # Construct the product dictionary
        product = {
            "Brand": brand_text,
            "Title": title_text,
            "Category": category,
            "Price": price_text,
            "Count": count,
            "Quantity": w_v_text,
            "Unit": unit,
            "Image Link": list(image_links),
            "Url":url_page
        }

        product.update(details)
        return product

    except Exception as e:
        print(f"Error: {e} while processing {url_page}")
        return None
    

# Function to handle multithreading
def scrap_product_concurrently(product_links):
    product_details = []
    with ThreadPoolExecutor(max_workers=2) as executor:  # Adjust max_workers as needed
        future_to_url = {executor.submit(scrap_product, url): url for url in product_links}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                product = future.result()
                if product:
                    product_details.append(product)
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')
    return product_details

# Main function
if __name__ == '__main__':
    category = search_category()
    print(category)
    product_detail = []
    if category:
        product_links = scrap_category(category)
        print(f"Found {len(product_links)} products.")
        if product_links:
            product_detail = scrap_product_concurrently(product_links)
    else:
        print("No category found")

    if product_detail:
        keys = product_detail[0].keys()
        with open('bigbasket_data.csv', 'a', newline='', encoding='utf-8') as output_file:
            dict_writer = csv.DictWriter(output_file, fieldnames=keys)
            # dict_writer.writeheader()  # Uncomment if writing the header for the first time
            dict_writer.writerows(product_detail)
    else:
        print("No product found")

    print("Processing done")

driver.quit()

 beverages


beverages
Found 2716 products.
